In [39]:
# install Spark

import os

# find latest version of spark
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Install Postgres Driver
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (87.6 kB/s)
Reading package li

In [40]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-10-10 01:47:22--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.06MB/s    in 0.8s    

2022-10-10 01:47:24 (1.06 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [41]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [42]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [43]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])

In [44]:
filter_total_votes_df = vine_df.filter(vine_df['total_votes']>=20)

In [45]:
filter_helpful_votes_df = filter_total_votes_df.filter(
    (filter_total_votes_df['helpful_votes'])/(filter_total_votes_df['total_votes'])>=0.50
    )

In [46]:
# split into vine members and non-members
vine_members_df = filter_helpful_votes_df.filter(filter_helpful_votes_df['vine']=='Y')
non_vine_members_df = filter_helpful_votes_df.filter(filter_helpful_votes_df['vine']=='N')

In [47]:
from pyspark.sql.functions import sum,count

# total votes for vine members and non-members
total_vine_members_reviews = vine_members_df.count()
total_non_vine_members_reviews = non_vine_members_df.count()

# total 5-star review votes for vine members and non-members
vine_members_5star = vine_members_df.filter(vine_members_df['star_rating']==5).count()
non_vine_members_5star = non_vine_members_df.filter(non_vine_members_df['star_rating']==5).count()

# percent 5 star reviews for vine members and non-members
vine_members_5star_percent = (vine_members_5star/total_vine_members_reviews)*100
non_vine_members_5star_percent = (non_vine_members_5star/total_non_vine_members_reviews)*100

results = {
    'Total Vine Member Reviews': total_vine_members_reviews,
    'Total Vine Member 5 Star Reviews': vine_members_5star,
    'Percent Vine Member 5 Star Reviews': vine_members_5star_percent,

    'Total Non-Vine Member Reviews': total_non_vine_members_reviews,
    'Total Non-Vine Member 5 Star Reviews': non_vine_members_5star,
    'Percent Non-Vine Member 5 Star Reviews': non_vine_members_5star_percent
}

In [48]:
results

{'Total Vine Member Reviews': 386,
 'Total Vine Member 5 Star Reviews': 176,
 'Percent Vine Member 5 Star Reviews': 45.59585492227979,
 'Total Non-Vine Member Reviews': 48717,
 'Total Non-Vine Member 5 Star Reviews': 24026,
 'Percent Non-Vine Member 5 Star Reviews': 49.317486708951705}